## Expression (∣0⟩⟨0∣⊗I+∣1⟩⟨1∣⊗σx][cos(t)I⊗I+isin(t)I⊗σz​][(∣0⟩⟨0∣⊗I+∣1⟩⟨1∣⊗σx]

In [1]:
import sympy as sp
import numpy as np

def main():
    # Define symbols
    t = sp.Symbol('t', real=True)
    i = sp.I

    # Define the matrices
    ket0bra0 = sp.Matrix([[1, 0], [0, 0]])
    ket1bra1 = sp.Matrix([[0, 0], [0, 1]])
    I = sp.eye(2)  # Identity matrix
    sigma_x = sp.Matrix([[0, 1], [1, 0]])
    sigma_z = sp.Matrix([[1, 0], [0, -1]])

    # Define the tensor products
    ket0bra0_tensor_I = sp.tensorproduct(ket0bra0, I)
    ket1bra1_tensor_sigma_x = sp.tensorproduct(ket1bra1, sigma_x)
    
    term1 = ket0bra0_tensor_I + ket1bra1_tensor_sigma_x
    term2 = sp.cos(t) * sp.tensorproduct(I, I) + i * sp.sin(t) * sp.tensorproduct(I, sigma_z)
    term3 = ket0bra0_tensor_I + ket1bra1_tensor_sigma_x

    # Compute the full expression
    result = np.matmul(term1,term2)
    print(type(result))
    print(type(term1))
    print("Term 1 Matrix:")
    sp.pprint(term1, use_unicode=True)

    print("\nTerm 2 Matrix:")
    sp.pprint(term2, use_unicode=True)

    print("\nTerm 3 Matrix:")
    sp.pprint(term3, use_unicode=True)
    
    print("\nExpanded Expression Result:")
    sp.pprint(result, use_unicode=True)

if __name__ == "__main__":
    main()


<class 'numpy.ndarray'>
<class 'sympy.tensor.array.dense_ndim_array.ImmutableDenseNDimArray'>
Term 1 Matrix:
⎡⎡1  0⎤  ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥⎥
⎢⎣0  1⎦  ⎣0  0⎦⎥
⎢              ⎥
⎢⎡0  0⎤  ⎡0  1⎤⎥
⎢⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣1  0⎦⎦

Term 2 Matrix:
⎡⎡ⅈ⋅sin(t) + cos(t)          0         ⎤                  ⎡0  0⎤              
⎢⎢                                     ⎥                  ⎢    ⎥              
⎢⎣        0          -ⅈ⋅sin(t) + cos(t)⎦                  ⎣0  0⎦              
⎢                                                                             
⎢                ⎡0  0⎤                   ⎡ⅈ⋅sin(t) + cos(t)          0       
⎢                ⎢    ⎥                   ⎢                                   
⎣                ⎣0  0⎦                   ⎣        0          -ⅈ⋅sin(t) + cos(

   ⎤
   ⎥
   ⎥
   ⎥
  ⎤⎥
  ⎥⎥
t)⎦⎦

Term 3 Matrix:
⎡⎡1  0⎤  ⎡0  0⎤⎤
⎢⎢    ⎥  ⎢    ⎥⎥
⎢⎣0  1⎦  ⎣0  0⎦⎥
⎢              ⎥
⎢⎡0  0⎤  ⎡0  1⎤⎥
⎢⎢    ⎥  ⎢    ⎥⎥
⎣⎣0  0⎦  ⎣1  0⎦⎦

Expanded Expression Result:
  [

In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import X, Z, H, I

# Define the matrices
ket0bra0 = np.Matrix([[1, 0], [0, 0]])
ket1bra1 = np.Matrix([[0, 0], [0, 1]])
# I = sp.eye(2)  # Identity matrix
sigma_x = np.Matrix([[0, 1], [1, 0]])
# sigma_z = sp.Matrix([[1, 0], [0, -1]])



def get_single_qubit_matrix(gate_name, qubit_index, n_qubits):
    """
    Get the matrix representation of a single-qubit gate in an n-qubit system.
    """
    gate = None
    if gate_name == 'X':
        gate = X()
    elif gate_name == 'Z':
        gate = Z()
    elif gate_name == 'H':
        gate = H()
    else:
        raise ValueError(f"Unsupported single-qubit gate: {gate_name}")

    # Convert to matrix
    gate_matrix = gate.to_matrix()

    # Tensor with identity matrices
    identity = np.eye(2)
    term_matrix = gate_matrix
    for i in range(n_qubits):
        if i != qubit_index:
            term_matrix = np.kron(term_matrix, identity)
    
    return term_matrix

def get_cnot_matrix(control_qubit, target_qubit, n_qubits):
    """
    Get the matrix representation of a CNOT gate in an n-qubit system.
    """
    # Create the identity matrix for 2 qubits
    I4 = np.eye(4)

    # Define the CNOT gate operation
    CNOT = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]])

    # Expand the CNOT gate to n-qubits
    term_matrix = np.eye(2**n_qubits)
    # Tensor with identity matrices
    for i in range(n_qubits):
        if i == control_qubit:
            term_matrix = np.kron(term_matrix, I4)
        elif i == target_qubit:
            term_matrix = np.kron(term_matrix, CNOT)
        else:
            term_matrix = np.kron(term_matrix, I4)

    return term_matrix

def calculate_matrix(circuit: QuantumCircuit) -> np.ndarray:
    """
    Calculate the unitary matrix of a given quantum circuit.
    
    Parameters:
    - circuit (QuantumCircuit): A Qiskit QuantumCircuit object.

    Returns:
    - np.ndarray: The unitary matrix of the input circuit.
    """
    n_qubits = circuit.num_qubits
    matrices = []

    for gate, qubits, params in circuit.data:
        if gate.name in ['x', 'y', 'z', 'h', 's', 't']:
            # Single-qubit gates
            matrices.append(get_single_qubit_matrix(gate.name.upper(), qubits[0].index, n_qubits))
        elif gate.name == 'cx':
            # CNOT gates
            control_qubit, target_qubit = qubits
            matrices.append(get_cnot_matrix(control_qubit.index, target_qubit.index, n_qubits))

    # Multiply all matrices together
    unitary_matrix = np.eye(2**n_qubits)
    for matrix in matrices:
        unitary_matrix = np.matmul(unitary_matrix, matrix)

    return unitary_matrix

# Example usage:
if __name__ == "__main__":
    # Create a quantum circuit with 4 qubits
    qc = QuantumCircuit(4)
    
    # Apply a Z gate on qubit 0
    qc.z(0)
    
    # Apply CNOT gates
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    
    # Calculate the unitary matrix of the circuit
    unitary_matrix = calculate_matrix(qc)
    
    print("Unitary matrix of the circuit:")
    print(unitary_matrix)


ImportError: cannot import name 'X' from 'qiskit.circuit.library' (/opt/anaconda3/lib/python3.12/site-packages/qiskit/circuit/library/__init__.py)

In [3]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, ZGate, HGate

def get_single_qubit_matrix(gate_name, qubit_index, n_qubits):
    """
    Get the matrix representation of a single-qubit gate in an n-qubit system.
    """
    # Define gate matrices
    gate_matrices = {
        'X': np.array([[0, 1], [1, 0]]),  # X gate (Pauli-X or NOT gate)
        'Z': np.array([[1, 0], [0, -1]]), # Z gate (Pauli-Z)
        'H': np.array([[1, 1], [1, -1]]) / np.sqrt(2), # Hadamard gate
    }

    if gate_name not in gate_matrices:
        raise ValueError(f"Unsupported single-qubit gate: {gate_name}")

    gate_matrix = gate_matrices[gate_name]

    # Tensor with identity matrices
    identity = np.eye(2)
    term_matrix = gate_matrix
    for i in range(n_qubits):
        if i != qubit_index:
            term_matrix = np.kron(term_matrix, identity)
    
    return term_matrix

def get_cnot_matrix(control_qubit, target_qubit, n_qubits):
    """
    Get the matrix representation of a CNOT gate in an n-qubit system.
    """
    # Define the CNOT gate matrix in a 4-qubit system
    CNOT = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]])

    # Create identity matrices for the other qubits
    identity = np.eye(2)
    term_matrix = np.eye(2**n_qubits)
    
    for i in range(n_qubits):
        if i == control_qubit:
            term_matrix = np.kron(term_matrix, identity)
        elif i == target_qubit:
            term_matrix = np.kron(term_matrix, CNOT)
        else:
            term_matrix = np.kron(term_matrix, identity)
    
    return term_matrix

def calculate_matrix(circuit: QuantumCircuit) -> np.ndarray:
    """
    Calculate the unitary matrix of a given quantum circuit.
    
    Parameters:
    - circuit (QuantumCircuit): A Qiskit QuantumCircuit object.

    Returns:
    - np.ndarray: The unitary matrix of the input circuit.
    """
    n_qubits = circuit.num_qubits
    matrices = []

    for gate, qubits, params in circuit.data:
        if gate.name in ['x', 'y', 'z', 'h', 's', 't']:
            # Single-qubit gates
            matrices.append(get_single_qubit_matrix(gate.name.upper(), qubits[0].index, n_qubits))
        elif gate.name == 'cx':
            # CNOT gates
            control_qubit, target_qubit = qubits
            matrices.append(get_cnot_matrix(control_qubit.index, target_qubit.index, n_qubits))

    # Multiply all matrices together
    unitary_matrix = np.eye(2**n_qubits)
    for matrix in matrices:
        unitary_matrix = np.matmul(unitary_matrix, matrix)

    return unitary_matrix

# Example usage:
if __name__ == "__main__":
    # Create a quantum circuit with 4 qubits
    qc = QuantumCircuit(4)
    
    # Apply a Z gate on qubit 0
    qc.z(0)
    
    # Apply CNOT gates
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    
    # Calculate the unitary matrix of the circuit
    unitary_matrix = calculate_matrix(qc)
    
    print("Unitary matrix of the circuit:")
    print(unitary_matrix)


/var/folders/6g/7pb_sfy561v161pgxcn9wdch0000gn/T/ipykernel_3139/1218641989.py:67: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for gate, qubits, params in circuit.data:


AttributeError: 'Qubit' object has no attribute 'index'